<a href="https://colab.research.google.com/github/sightes/Scraping/blob/main/diartek_cl_scrapping_web_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El presente codigo realiza un scrapping completo de los productos cargados en diartek.cl, tener cuidado con los precios dado que existen algunos q se encuentran en dolares sin embargo en el sitio web no existe referencia a la moneda

In [1]:
%%capture
!pip install selenium
!pip install fake_useragent

In [16]:
import pandas as pd
import json
from selenium.webdriver.common.by import By
from selenium import webdriver
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import numpy as np


def try_or(func, default=None, expected_exc=(Exception,)):
    try:
        return func()
    except expected_exc:
        return default

def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    #options.add_argument("--incognito")
    options.add_argument("--nogpu")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1280,1280")
    options.add_argument("--no-sandbox")
    options.add_argument("--user-data-dir=chrome-data")
    options.add_argument("--enable-javascript")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
    ua = UserAgent()
    userAgent = ua.random
    driver = webdriver.Chrome(options=options)
    driver.execute_cdp_cmd('Network.setUserAgentOverride',{"userAgent": userAgent,'method': 'GET','Host':'www.solili.mx'})
    driver.set_network_conditions(offline=False,
                                  latency=5,
                                  download_throughput=500 * 1024,
                                  upload_throughput=500 * 1024)
    return(driver)
def get_atributes(a):
  return([float(a.get_attribute('data-price').replace('.','').replace(',','.').replace('$','')),
  float(a.get_attribute('data-price-default').replace('.','').replace(',','.').replace('$','')),
  a.get_attribute('data-brand'),
  a.get_attribute('data-name'),
  'https://diartek.cl/'+a.get_attribute('data-permalink'),
  a.get_attribute('data-image'),
  a.get_attribute('data-id')])


In [3]:
driver=create_driver()
driver.get('https://diartek.cl/')
menus=driver.find_elements(By.XPATH,"//*[contains(@class,'accordion-button collapsed sidebar-accordion__button sidebar-accordion__button--inner')]")
links=[i.get_attribute('href') for i in menus]
driver.close()
selected_links=[]
for i in links:
  if len(i.split('/'))<5:
    selected_links=selected_links+[i]

In [17]:
productos=[]
for c_link in selected_links:
  driver=create_driver()
  driver.get(c_link)
  max_page=try_or(lambda: int(driver.find_elements(By.XPATH,"//*[contains(text(),'Último')]")[0].get_attribute('href').split('=')[1]), default=0)
  c_productos=driver.find_elements(By.XPATH,"//div[contains(@class,'product-block__actions')]/form")
  productos=productos+[get_atributes(i) for i in c_productos]
  if max_page>1:
    for i_page in range(2,max_page+1):
      c_link=c_link+'?page='+str(i_page)
      driver.get(c_link)
      c_productos=driver.find_elements(By.XPATH,"//div[contains(@class,'product-block__actions')]/form")
      productos=productos+[get_atributes(i) for i in c_productos]
  driver.close()

In [19]:
pd.DataFrame(productos).to_csv('data.csv')